## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,-9.38,70,100,4.61,overcast clouds
1,1,Buala,SB,-8.1450,159.5921,77.41,92,100,5.64,moderate rain
2,2,Busselton,AU,-33.6500,115.3333,57.16,77,57,8.63,broken clouds
3,3,Torbay,CA,47.6666,-52.7314,40.75,93,100,5.75,light rain
4,4,Honningsvag,NO,70.9821,25.9704,31.75,64,40,25.32,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 44
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Buala,SB,-8.1450,159.5921,77.41,92,100,5.64,moderate rain
2,2,Busselton,AU,-33.6500,115.3333,57.16,77,57,8.63,broken clouds
5,5,Tres Arroyos,AR,-38.3739,-60.2798,71.40,28,70,18.72,broken clouds
6,6,Belyy Yar,RU,53.6039,91.3903,53.49,74,0,4.47,clear sky
7,7,Port Hardy,CA,50.6996,-127.4199,44.60,87,75,9.22,light intensity shower rain
10,10,Mataura,NZ,-46.1927,168.8643,51.37,97,100,8.81,overcast clouds
12,12,Bavaniste,RS,44.8189,20.8758,64.85,31,0,4.21,clear sky
13,13,Ndele,CF,8.4109,20.6473,87.44,41,97,5.59,overcast clouds
14,14,Bluff,NZ,-46.6000,168.3333,54.97,76,100,28.30,overcast clouds
15,15,Cape Town,ZA,-33.9258,18.4232,65.10,73,75,12.66,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                436
City                   436
Country                436
Lat                    436
Lng                    436
Max Temp               436
Humidity               436
Cloudiness             436
Wind Speed             436
Current Description    436
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Buala,SB,77.41,moderate rain,-8.1450,159.5921,
2,Busselton,AU,57.16,broken clouds,-33.6500,115.3333,
5,Tres Arroyos,AR,71.40,broken clouds,-38.3739,-60.2798,
6,Belyy Yar,RU,53.49,clear sky,53.6039,91.3903,
7,Port Hardy,CA,44.60,light intensity shower rain,50.6996,-127.4199,
10,Mataura,NZ,51.37,overcast clouds,-46.1927,168.8643,
12,Bavaniste,RS,64.85,clear sky,44.8189,20.8758,
13,Ndele,CF,87.44,overcast clouds,8.4109,20.6473,
14,Bluff,NZ,54.97,overcast clouds,-46.6000,168.3333,
15,Cape Town,ZA,65.10,light rain,-33.9258,18.4232,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Buala,SB,77.41,moderate rain,-8.1450,159.5921,Maringe Lagoon Lodge
2,Busselton,AU,57.16,broken clouds,-33.6500,115.3333,Observatory Guest House
5,Tres Arroyos,AR,71.40,broken clouds,-38.3739,-60.2798,Habitacion Triple con baño y entrada privada
6,Belyy Yar,RU,53.49,clear sky,53.6039,91.3903,Pasportnyy Stol
7,Port Hardy,CA,44.60,light intensity shower rain,50.6996,-127.4199,Airport Inn-Port Hardy
...,...,...,...,...,...,...,...
664,Turangi,NZ,57.58,overcast clouds,-39.0000,175.9333,
665,Alcains,PT,58.55,overcast clouds,39.9164,-7.4566,Casa do Balcão
666,Dumas,US,59.14,clear sky,35.8656,-101.9732,"Holiday Inn Express & Suites Dumas, an IHG Hotel"
667,Labuhan,ID,79.63,overcast clouds,-6.8844,112.2051,DoubleFAG


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Buala,SB,77.41,moderate rain,-8.1450,159.5921,Maringe Lagoon Lodge
2,Busselton,AU,57.16,broken clouds,-33.6500,115.3333,Observatory Guest House
5,Tres Arroyos,AR,71.40,broken clouds,-38.3739,-60.2798,Habitacion Triple con baño y entrada privada
6,Belyy Yar,RU,53.49,clear sky,53.6039,91.3903,Pasportnyy Stol
7,Port Hardy,CA,44.60,light intensity shower rain,50.6996,-127.4199,Airport Inn-Port Hardy
...,...,...,...,...,...,...,...
662,Simontornya,HU,56.61,broken clouds,46.7546,18.5549,Fried Kastélyszálló Resort
665,Alcains,PT,58.55,overcast clouds,39.9164,-7.4566,Casa do Balcão
666,Dumas,US,59.14,clear sky,35.8656,-101.9732,"Holiday Inn Express & Suites Dumas, an IHG Hotel"
667,Labuhan,ID,79.63,overcast clouds,-6.8844,112.2051,DoubleFAG


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))